In [1]:
# Data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# The figure will be rendered inline a my Jupyter Notebook
output_notebook()

In [ ]:
df = pd.read_csv('C:/Users/isabe/Documents/GitHub/CWF_Fall2020/data/fan duel/fd_salaries_2019.csv')
df.head()

In [ ]:
# Removing lines with salary = 0
remove_sal = df['fd_salary'] != 0
df = df[remove_sal]

In [ ]:
# Removing NaN 
df = df.dropna()

In [ ]:
source = ColumnDataSource(df)

In [ ]:
# Using matplot
plt.figure(figsize=(10,7))
plt.bar(avg_points_per_position.index,avg_points_per_position.values, color='cyan', alpha=0.4)
plt.title('Average Number of points by position')
plt.xlabel('Position')
plt.ylabel('Average number of points')
plt.show()